In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

class DeepQNetwork(nn.Module):
    def __init__(self, learning_rate):
        super(DeepQNetwork, self).__init__()
        
        self.conv1 = nn.Conv2d(1,32,8, stride=4,padding=1)
        self.conv2 = nn.Conv2d(32,64,4, stride=2)
        self.conv3 = nn.Conv2d(64, 128, 3)
        
        #19 is just trial and error
        self.dense1 = nn.Linear(128*19*8, 512) 
        self.dense2 = nn.Linear(512,6) #6 actions (nothing is 1)
        
        self.optimizer = optim.RMSprop(self.parameters(),
                                       lr=learning_rate)
        self.loss = nn.MSELoss()
        
        #change device to gpu if possible
        self.device = torch.device('cpu')
        self.to(self.device)
        
    def forward(self, observation):
        observation = T.Tensor(observation).to(self.device)
        observation = observation.view(-1, 1, 185, 95)
        observation = F.relu(self.conv1(observation))
        observation = F.relu(self.conv2(observation))
        observation = F.relu(self.conv3(observation))
        #now flatten
        observation = observation.view(-1,128*19*8)
        #now the dense part 
        observation = F.relu(self.dense1(observation))
        actions = F.relu(self.dense2(observation))
        
        return actions

In [ ]:
class Agent(object):
    def __init__(self,discount,random_move_frequency,
                 learning_rate,max_memory_size,
                 min_random_ove=.05,
                 target_network_udate_frequency=10000, 
                 action_space=[0,1,2,3,4,5]):
        self.discount = discount
        self.random_move_frequency = 